In [110]:
# open and read input file
with open("kendrick_lamar_lyrics.txt", "r", encoding="utf-8") as f:
    lyrics = f.read()

In [111]:
# check how many characters there are in the dataset
len(lyrics)

554065

In [112]:
# take a look at the first 1000 characters
print(lyrics[:1000])

sit still and close your eyes smoke to it
what's behind the other door oh-ohh
no more silence no more silence
don't kill this thing we got called love don't shoot
just searching for the perfect shot

i used to write rhymes all day and all night
when y'all was playing playstation, my pencil was erasing lines
my conscience only knew what's half-tight
at 3:14, it's time to get me a slice my nigga
this is a dog's fight my nigga
the soundtrack to life my nigga
kendrick lamar, his momma called him that
he watched house party and ate apple jacks
he sold sega games, his cousin sold crack
he pumped reeboks, his uncles pumped packs
punk fake, jump-shot, ball hit the back
ball dreams of being point guard was off limits jack
that's because these compton streets was built not to win
you killed the nigga, i stole a bible, is that a sin
part of me though, i'm searching for answers just searching for the perfect shot
the good kid from the ugly city that's mad that he's had some
where is the love

when

In [113]:
# check all the unique characters that occur in the dataset
chars = sorted(list(set(lyrics)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !"&'(),-./0123456789:?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzàâãéíïúе– 
86


## Simple tokeniser for the text/lyrics


In [114]:
# create a mapping from characters to integers (encoder) and vice-versa (decoder)
str_to_int = {char: i for i, char in enumerate(chars)}
int_to_str = {i: char for i, char in enumerate(chars)}

In [115]:
# encoder takes a string which outputs a list of integers ,
# characters in string are converted to int via lookup table
encode = lambda s: [str_to_int[c] for c in s]

# decoder takes a list of integers and outputs a string, integers converted via lookup table
decode = lambda l: "".join([int_to_str[i] for i in l])

In [116]:
print(encode("helloo there"))
print(decode(encode("helloo there")))

[57, 54, 61, 61, 64, 64, 1, 69, 57, 54, 67, 54]
helloo there


In [117]:
# encode the entire dataset from string to integers
import torch

data = torch.tensor(encode(lyrics), dtype=torch.long)
print(data.shape, data.type)
# the first 1000 characters from our dataset converted using our encoder mapping (from str to int)
print(data[:1000])

torch.Size([554065]) <built-in method type of Tensor object at 0x0000018AD497BE30>
tensor([68, 58, 69,  1, 68, 69, 58, 61, 61,  1, 50, 63, 53,  1, 52, 61, 64, 68,
        54,  1, 74, 64, 70, 67,  1, 54, 74, 54, 68,  1, 68, 62, 64, 60, 54,  1,
        69, 64,  1, 58, 69,  0, 72, 57, 50, 69,  5, 68,  1, 51, 54, 57, 58, 63,
        53,  1, 69, 57, 54,  1, 64, 69, 57, 54, 67,  1, 53, 64, 64, 67,  1, 64,
        57,  9, 64, 57, 57,  0, 63, 64,  1, 62, 64, 67, 54,  1, 68, 58, 61, 54,
        63, 52, 54,  1, 63, 64,  1, 62, 64, 67, 54,  1, 68, 58, 61, 54, 63, 52,
        54,  0, 53, 64, 63,  5, 69,  1, 60, 58, 61, 61,  1, 69, 57, 58, 68,  1,
        69, 57, 58, 63, 56,  1, 72, 54,  1, 56, 64, 69,  1, 52, 50, 61, 61, 54,
        53,  1, 61, 64, 71, 54,  1, 53, 64, 63,  5, 69,  1, 68, 57, 64, 64, 69,
         0, 59, 70, 68, 69,  1, 68, 54, 50, 67, 52, 57, 58, 63, 56,  1, 55, 64,
        67,  1, 69, 57, 54,  1, 65, 54, 67, 55, 54, 52, 69,  1, 68, 57, 64, 69,
         0,  0, 58,  1, 70, 68, 54, 5

## Create our train and validation datasets


In [118]:
# split 90% of our data into train and have remaining 10% as our validation
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [119]:
len(train_data)

498658

In [120]:
len(val_data)

55407

In [121]:
# use block_size / context length interchangeably
block_size = 8
train_data[: block_size + 1]

tensor([68, 58, 69,  1, 68, 69, 58, 61, 61])

given a tensor above, there are 8 training examples within a sample of size blocksize. <br>
E.g : <br>
x = 68 , y = 58 <br>
x = [68,58], y = 69 <br>
x = [68,58,69] , y = 1 <br>
etc until the 8th training example within this sample <br>
x = [68,58,69,1,68,69,58,61], y = 61 <br>


In [122]:
x = train_data[:block_size]
# y is technically our target for the input of x, thus it is ofset by 1
y = train_data[1 : block_size + 1]

In [123]:
x

tensor([68, 58, 69,  1, 68, 69, 58, 61])

In [124]:
y

tensor([58, 69,  1, 68, 69, 58, 61, 61])

In [125]:
for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"when in put is {context} the target is: {target}")

when in put is tensor([68]) the target is: 58
when in put is tensor([68, 58]) the target is: 69
when in put is tensor([68, 58, 69]) the target is: 1
when in put is tensor([68, 58, 69,  1]) the target is: 68
when in put is tensor([68, 58, 69,  1, 68]) the target is: 69
when in put is tensor([68, 58, 69,  1, 68, 69]) the target is: 58
when in put is tensor([68, 58, 69,  1, 68, 69, 58]) the target is: 61
when in put is tensor([68, 58, 69,  1, 68, 69, 58, 61]) the target is: 61


In [126]:
torch.manual_seed(42)
batch_size = 4  # how many independent sequences will we process in parallel
block_size = 8  # what is the maximum context length for predictions

In [127]:
def get_batch(split):
    # generate a small batch of data for inputs x and targets y
    data = train_data if split == "train" else val_data

    # generate a tensor of random integers, that represent the starting position of each sequence of data
    batch_start_indicies = torch.randint(high=(len(data) - block_size), size=(batch_size,))

    # creates a tensor: x and y where each element is a sequence of block_size, stack the 1-D tensors as rows
    # creating a batch_size x block_size tensors (e.g 4x8 tensor)
    x = torch.stack([data[index : index + block_size] for index in batch_start_indicies])
    y = torch.stack([data[index + 1 : index + block_size + 1] for index in batch_start_indicies])
    return x, y

In [128]:
xb, yb = get_batch('train')
print(f"Batch size = {batch_size} \nBlock size = {block_size} \n")

print('inputs: ')
print(xb.shape)
print(xb, '\n')

print('targets: ')
print(yb.shape)
print(yb, '\n')


Batch size = 4 
Block size = 8 

inputs: 
torch.Size([4, 8])
tensor([[58, 52, 57,  1, 63, 58, 56, 56],
        [64, 72,  1, 59, 70, 68, 69,  1],
        [55, 70, 52, 60, 58, 63,  5,  1],
        [32,  5, 62,  1, 51, 61, 54, 68]]) 

targets: 
torch.Size([4, 8])
tensor([[52, 57,  1, 63, 58, 56, 56, 50],
        [72,  1, 59, 70, 68, 69,  1, 72],
        [70, 52, 60, 58, 63,  5,  1, 62],
        [ 5, 62,  1, 51, 61, 54, 68, 68]]) 



In [129]:
for b in range(batch_size): # iterate through the batch dimension (batches)
    for t in range(block_size): # iterate through the time dimension (each character/element/number in the batch)
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} the target is {target}")
    print("\n")

When input is [58] the target is 52
When input is [58, 52] the target is 57
When input is [58, 52, 57] the target is 1
When input is [58, 52, 57, 1] the target is 63
When input is [58, 52, 57, 1, 63] the target is 58
When input is [58, 52, 57, 1, 63, 58] the target is 56
When input is [58, 52, 57, 1, 63, 58, 56] the target is 56
When input is [58, 52, 57, 1, 63, 58, 56, 56] the target is 50


When input is [64] the target is 72
When input is [64, 72] the target is 1
When input is [64, 72, 1] the target is 59
When input is [64, 72, 1, 59] the target is 70
When input is [64, 72, 1, 59, 70] the target is 68
When input is [64, 72, 1, 59, 70, 68] the target is 69
When input is [64, 72, 1, 59, 70, 68, 69] the target is 1
When input is [64, 72, 1, 59, 70, 68, 69, 1] the target is 72


When input is [55] the target is 70
When input is [55, 70] the target is 52
When input is [55, 70, 52] the target is 60
When input is [55, 70, 52, 60] the target is 58
When input is [55, 70, 52, 60, 58] the targ

Our 4x8 tensor contains 32 training examples, which are completely independent

In [130]:
# our input to the transformer 
xb

tensor([[58, 52, 57,  1, 63, 58, 56, 56],
        [64, 72,  1, 59, 70, 68, 69,  1],
        [55, 70, 52, 60, 58, 63,  5,  1],
        [32,  5, 62,  1, 51, 61, 54, 68]])

In [131]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(42)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        # maps each token in the sequence to its next-token prediction in the form of logits
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensors of integers
        logits = self.token_embedding_table(
            idx
        )  # (B,T,C) , where C = channel = vocab size

        if targets is None:
            # if no target, just get the logits
            loss = None
        else:
            # since we have a multi-dimensional input, PyTorch cross_entropy function expects the logit tensor shape to be (B*T,C) and not (B,T,C)
            # and the target tensor to be 1D. Therefore we have to reshape our logit and target tensors
            B, T, C = logits.shape
            logits = logits.view(B * T, C) # want it to be 2-D Tensor
            targets = targets.view(B*T)  # want it to be 1-D Tensor

            # measures the quality of the logits w.r.t the targets,
            # how well are we predicting the next character based on the logits
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) tensor of indicies in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            # only consider last element in the time dimension, becomes (B,C)
            logits = logits[:,-1,:] 
            # apply softmax to get proababilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution, only gets 1 prediction so becomes (B,1)
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [132]:
model = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = model(xb,yb)


In [133]:
logits.shape

torch.Size([32, 86])

In [134]:
loss

tensor(4.9753, grad_fn=<NllLossBackward0>)

we are expecting a loss of -ln(1/vocab_size)

In [135]:
import math
from math import log
print(f"We expect the loss to be about {-log(1/vocab_size)}")

We expect the loss to be about 4.454347296253507


In [136]:
idx = torch.zeros((1, 1), dtype=torch.long)

print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))


í–v L'szïU.е7/bR
Z1g6'sl)HwPp(–"âXo//Dâ?JIwNGqígàl(ZéG-YY4a6A9е,Oú–CàKJ3")LNkm1é9ASpL/qKJk/D(WdfB234


generated output is random because our model hasnt been trained

## Model Training

In [137]:
# intialise optimiser
optimiser = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [138]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate loss
    logits, loss = model(xb,yb)
    optimiser.zero_grad(set_to_none=True)
    loss.backward()
    optimiser.step()

print(loss.item())

2.471187114715576


In [139]:
idx = torch.zeros((1, 1), dtype=torch.long)

print(decode(model.generate(idx, max_new_tokens=500)[0].tolist()))


te yof is theaYSAl(Msin ache fomancenet'mery gand l n boucanarr g, m

iedole
i g m g, bul e
id sithe joren ccr sthandovorlorall
iman ly ait t, me mp, m ite intul me, bl tcourinck
but my, tot jus bas a tht h m ton'm t icillundy gil daroma juc gif, as
atinith a t bewery dediali wheambugumy zis tien'sthatheretherkel wige itanndpeledorigen wi-Rt hemine RUpodbr c mey hino aut peg ithed a lor, gachortin ilache, piler o
An'aixts buloon, thur o hearomyolala yontwathtoverbeen dne t anke?44
titr bean'myou
